In [1]:
# data manipulation
import pandas as pd
import numpy as np

# model training
from sklearn.model_selection import GridSearchCV

# classifiers
from sklearn.naive_bayes import GaussianNB # naive bayes
from sklearn.neighbors import KNeighborsClassifier # KNN
from sklearn.svm import SVC # SVM
from sklearn.ensemble import BaggingClassifier 
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier

# machine learning models/techniques third parties 
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import sklearn.linear_model as sklm

# for final validation to plot ROC curve
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc

%matplotlib inline

from sklearn.model_selection import cross_val_score
from sklearn import preprocessing

from tensorflow import set_random_seed
import keras
from keras.models import Model
from keras.layers import Input, Conv2D, Dense, Activation, MaxPool2D, Dropout, Flatten
from keras import optimizers, losses
from keras.models import model_from_json


# to time experiments
import time


# to calculate kurtosis and skew
from scipy.stats import kurtosis
from scipy.stats import skew

# for plots
import seaborn as sns 
import matplotlib as pl
import matplotlib.pyplot as plt

import dataexp as dtex # our code
import dataio as dtio # our code 
import preprocessing as pre # our code

# to display pandas and other elements in html
from IPython.display import display
from IPython.core.display import HTML
from IPython.display import Image

# for ordered dictionaries 
from collections import OrderedDict

# for feature extraction
import feature_extraction as fe # our code

# machine learning models/techniques
import model_validation as me # our code
#from decision_tree import DecisionTree # our code
#from random_forest import RandomForest # our code
from logistic_regression import LogisticRegression # our code





# ignore warnings
import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


In [2]:
x_train_o = pd.read_csv('financial_data.csv')
y_train_o = pd.read_csv('revealed_businesses.csv')

x_test_o = pd.read_csv("testing_data.csv")

In [3]:
x_train_o.replace('?', np.nan, inplace=True)
x_train_o = x_train_o.astype('float64')


x_test_o.replace('?', np.nan, inplace=True)
x_test_o = x_test_o.astype('float64')

In [4]:
data_train_all = x_train_o.merge(y_train_o, on='Var1', how = 'left')


In [5]:
data_nolabel = data_train_all[data_train_all.Var66.isnull()]
data_label = data_train_all[data_train_all.Var66.notnull()]

In [6]:
data_nolabel.head()

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var8,Var9,Var10,...,Var57,Var58,Var59,Var60,Var61,Var62,Var63,Var64,Var65,Var66
1,15092.0,0.049699,0.065808,0.726800,12.94400,233.1100,0.00000,0.063192,14.19601,0.89618,...,0.059565,0.053189,0.93169,0.000000,5.0492,11.1520,24.784,14.7270,4.2204,NaN
3,14171.0,0.001417,0.708110,-0.052312,0.88978,-31.1980,0.26952,0.001407,0.41222,1.96540,...,-0.000535,0.004820,0.99930,0.745480,17.1011,7.9482,88.147,4.1408,3.4021,NaN
4,12900.0,0.020041,0.346520,0.335930,2.76130,39.0500,0.00000,0.020031,1.88591,1.29750,...,0.212410,0.030652,0.80158,0.000862,9.7670,6.7570,53.651,6.8032,2.7412,NaN
5,16499.0,0.140000,0.419930,0.290420,1.74160,7.1865,0.20062,0.177390,1.31961,1.10280,...,0.093230,0.252620,0.90677,0.051068,8.1696,5.1309,70.315,5.1909,6.3939,NaN
8,16122.0,0.094560,0.097575,0.422890,5.78220,82.3870,0.00000,0.108800,9.24851,1.27030,...,0.076875,0.104770,0.91846,0.000000,7.9041,4.5030,25.410,14.3650,2.5994,NaN


In [31]:
data_nolabel_v = data_nolabel.drop(columns=['Var1', 'Var66'])
data_nolabel_id = data_nolabel['Var1']

data_label_v = data_label.drop(columns=['Var1', 'Var66'])
data_label_id = data_label['Var1']

In [78]:
data_nolabel_v_f = data_nolabel_v.fillna(data_nolabel_v.mean())
data_label_v_f = data_label_v.fillna(data_label_v.mean())

In [36]:
data_nolabel_v_f = pd.DataFrame(preprocessing.StandardScaler().fit_transform(data_nolabel_v_f),columns = data_nolabel_v_f.columns)
data_label_v_f = pd.DataFrame(preprocessing.StandardScaler().fit_transform(data_label_v_f),columns = data_label_v_f.columns)

In [37]:
X_pred = data_nolabel_v_f.iloc[:,:].values
X_pred.shape

(4124, 64)

In [50]:
X = data_label_v_f.iloc[:,:].values
y = data_label['Var66'].values
X.shape

(4879, 64)

In [51]:
def eucliden(X, c):
    c = c.reshape(1, X.shape[1]) # 1 x p
    distances = np.sqrt(np.sum((X - c) ** 2, axis=1))
    return distances

In [ ]:
result = []
for test_x in X_pred:
    distances = eucliden(X, test_x)
    indexs = np.argsort(distances)[0:1]
    label = 1 if y[indexs].sum()>0 else 0
    result.append(label)

In [ ]:
y_pred = pd.DataFrame(result, columns=["Var66"])
y_pred['Var66'].value_counts()

In [95]:
#data_nolabel_selected_stand = pd.DataFrame(preprocessing.StandardScaler().fit_transform(data_nolabel_selected),columns = data_nolabel_selected.columns)
data_nolabel_selected_stand = data_nolabel_selected.reset_index().drop(columns=['index'])

In [96]:
data_nolabel_id_noidx = data_nolabel_id.reset_index()['Var1']
data_nolabel_v = data_nolabel_v.reset_index().drop(columns=['index'])

In [97]:

data_nolabel_pred = pd.concat([data_nolabel_id_noidx, data_nolabel_v_f, y_pred], axis=1)

In [98]:
data_nolabel_pred['Var66'].value_counts()

1    4123
0       1
Name: Var66, dtype: int64

In [99]:
data_label_fill = data_label.fillna(data_label.mean())

In [100]:
data_tran_all  = pd.concat([data_label_fill, data_nolabel_pred], axis=0)

In [101]:
data_tran_all.head()

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var8,Var9,Var10,...,Var57,Var58,Var59,Var60,Var61,Var62,Var63,Var64,Var65,Var66
0,18399.0,0.023954,0.150120,0.39567,3.63570,54.043,0.028822,0.031029,4.56831,1.01120,...,0.011041,0.034914,0.98896,0.000000,9.5214,5.8248,34.713,10.5150,3.4752,0.0
2,19821.0,-0.356310,0.392880,0.15884,1.40430,-2.619,-0.085597,-0.356320,1.54531,0.92963,...,-0.172770,-0.586910,1.38330,0.000000,5.6112,15.7790,154.260,2.3662,2.0738,0.0
6,17769.0,0.004100,0.723040,-0.47065,0.34907,-169.230,0.000000,0.004090,0.38306,1.10920,...,-0.096542,0.014766,0.99667,0.000000,15.8971,6.2881,237.920,1.5341,1.4837,0.0
7,19309.0,0.024596,0.084252,0.38657,5.58830,44.886,0.174290,0.028695,10.19801,1.00690,...,0.006825,0.028616,0.99317,0.000000,26.1531,15.2470,12.723,28.6870,4.5674,0.0
9,20728.0,0.036710,0.416930,0.13777,1.37330,-18.123,0.037957,0.050825,1.28881,0.99111,...,-0.008967,0.068297,1.00900,0.089127,7.6912,5.5223,84.191,4.3354,3.2441,0.0


In [102]:
data_nolabel_pred.isnull().sum().sum()

0

In [103]:
os_dfs_dict = OrderedDict()

os_dfs_dict['mean'] = pre.oversample_smote(data_tran_all, columns = data_tran_all.columns, verbose=True)

original dataset (labels): {0.0: 4710, 1.0: 4293}
total: 9003
resampled dataset (labels): {0.0: 4710, 1.0: 4710}
total: 9420



In [104]:
x_test_business_id = x_test_o['Var1']
x_test = x_test_o.drop(columns=['Var1'])
x_test_f = x_test.fillna(x_test.mean())

In [105]:
best_period = os_dfs_dict['mean'][0]
X = best_period.iloc[:,1:-1].values
y = best_period["Var66"].values

random_forest_classifier_sklearn_roc_all = RandomForestClassifier(criterion="gini", 
                                                          n_estimators = 10,
                                                          max_features = "sqrt", 
                                                          random_state = 94)

random_forest_classifier_sklearn_roc_all.fit(X, y)
y_pred_dt = random_forest_classifier_sklearn_roc_all.predict(x_test_f)


In [106]:
df_y = pd.DataFrame(y_pred_dt, columns=["Is_Bankrupted"])
upload = pd.concat([x_test_business_id, df_y], axis=1)
df_y["Is_Bankrupted"].value_counts()

1    1009
0     491
Name: Is_Bankrupted, dtype: int64

In [107]:
upload = upload.astype('int32')
upload.columns=['Business_ID', 'Is_Bankrupted']
upload.to_csv('3_22_1.csv', index=False)